In [ ]:
pip install ultralytics

In [8]:
import torch
import cv2
import os
from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
def load_yolov5_model(weights_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    return model

In [11]:
def draw_bounding_boxes(label_file,image_path, results):
    image = cv2.imread("/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg")
    boxes = results.xyxy[0].detach().cpu().numpy()

    with open(label_file, 'w') as f:
      for i,box in enumerate(boxes):
        print(i)
        im=image.copy()
        x1, y1, x2, y2 = box[:4].astype(int)
        coord=(x1,y1,x2,y2)
        im=im[y1:y2,x1:x2]
        os.chdir(image_path)
        cv2.imwrite(f"img{i}.jpg",im)
        os.chdir("/content/drive")
        print(coord)
        label = f"{results.names[int(box[5])]}"
        f.write(f"{x1} {y1} {x2} {y2}\n")

In [7]:
def load_model(model_path):
    model=YOLO(model=model_path)
    return model

In [18]:
def extract_roi(image_path,coord_list):
    image=cv2.imread(image_path)
    im=image.copy()

    x1, y1, x2, y2 = coord_list
    roi = im[y1:y2, x1:x2]
    # roi = image[y2:y1, x1:x2]
    return roi

In [23]:
def extract_labels(roi,model):

    img_dir="/content/drive/MyDrive/data/images/"
    img_list = os.listdir(img_dir)
    img_list.sort()
    image_path=img_dir + img_list[0]
    print(image_path)
    print(img_list[0])

    results=model.predict(source=roi,conf=0.5,save=False,show_labels=False)
    contours=[]
    # boxes=result.boxes
    for result in results:
        boxes = result.boxes

    i = 0
    label_string_list =[]
    num_objects=0
    num_emptys=0
    while i < len(boxes.xyxy.tolist()):

      bbox = boxes.xyxy.tolist()[i]
      class_id = int(boxes.cls.tolist()[i])
      if class_id==0:
        num_objects +=1
      else:
        num_emptys +=1
      i+=1
    return num_objects,num_emptys
    print("Objects:",num_objects)
    print("emptys:",num_emptys)

In [24]:
weights_path = "/content/drive/MyDrive/phase_2.pt"
image=cv2.imread("/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg")
model = load_yolov5_model(weights_path)
results=model(image)
image_path="/content/drive/MyDrive/data"
label_file="/content/drive/MyDrive/data/output"
draw_bounding_boxes(label_file,image_path, results)
model_path="/content/drive/MyDrive/best_detect.pt"
model=load_model(model_path)
results_dict={}
# output_path="/content/drive/MyDrive/data/output"
output_path=label_file
bounding_box_list=[]
with open(output_path, 'r') as f:
   for line in f:
    bounding_box = [int(coord) for coord in line.strip().split()]
    bounding_box_list.append(bounding_box)
destination_path = "/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg"
results_dict["image_name"] = destination_path.split("/")[-1]
results_dict["coordinates"]=bounding_box_list
counts_list=[]


for i in range(len(bounding_box_list)):
  roi = extract_roi(destination_path,bounding_box_list[i])
  num_objs,num_empts=extract_labels(roi,model)
  counts_list.append([num_objs,num_empts])
results_dict["count"] = counts_list
print(results_dict)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-25 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients
Adding AutoShape... 


0
(2859, 334, 4396, 1304)
1
(782, 1769, 1607, 2196)
2
(2875, 1448, 4471, 2443)
3
(2881, 2654, 4582, 3456)
4
(694, 2623, 1553, 3123)
5
(791, 705, 1634, 1487)
6
(1651, 1731, 2786, 2234)
7
(1, 2382, 613, 3039)
8
(1600, 2673, 2798, 3332)
9
(3, 1650, 718, 2250)
10
(70, 860, 771, 1534)


/content/drive/MyDrive/data/images/images0.jpg
images0.jpg


0: 416x640 4 objects, 149.8ms
Speed: 4.6ms preprocess, 149.8ms inference, 2.2ms postprocess per image at shape (1, 3, 416, 640)

0: 352x640 3 objects, 65.9ms
Speed: 2.9ms preprocess, 65.9ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)


/content/drive/MyDrive/data/images/images0.jpg
images0.jpg


...

/content/drive/MyDrive/data/images/images0.jpg
images0.jpg



0: 544x640 10 objects, 60.7ms
Speed: 4.7ms preprocess, 60.7ms inference, 2.4ms postprocess per image at shape (1, 3, 544, 640)


/content/drive/MyDrive/data/images/images0.jpg
images0.jpg



0: 640x640 14 objects, 16.7ms
Speed: 5.3ms preprocess, 16.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


/content/drive/MyDrive/data/images/images0.jpg
images0.jpg
{'image_name': 'Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg', 'coordinates': [[2859, 334, 4396, 1304], [782, 1769, 1607, 2196], [2875, 1448, 4471, 2443], [2881, 2654, 4582, 3456], [694, 2623, 1553, 3123], [791, 705, 1634, 1487], [1651, 1731, 2786, 2234], [1, 2382, 613, 3039], [1600, 2673, 2798, 3332], [3, 1650, 718, 2250], [70, 860, 771, 1534]], 'count': [[4, 0], [3, 0], [3, 0], [3, 0], [3, 0], [5, 0], [3, 0], [6, 0], [3, 0], [10, 0], [14, 0]]}
